In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd

dataset_path = "a_to_z_supermarket_transactions.csv"
data = pd.read_csv(dataset_path)

label_encoder_customer = LabelEncoder()
label_encoder_product = LabelEncoder()
label_encoder_category = LabelEncoder()

data['customer_id'] = label_encoder_customer.fit_transform(data['customer_id'])
data['product_id'] = label_encoder_product.fit_transform(data['product_id'])
data['category'] = label_encoder_category.fit_transform(data['category'])

X = data[['customer_id', 'product_id', 'category', 'quantity', 'price', 'total_spent']]
y = data['category']

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

model.save("a_to_z_supermarket_model.h5")
print("Model saved as 'a_to_z_supermarket_model.h5'")


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3127 - loss: 1.5832 - val_accuracy: 0.8150 - val_loss: 0.8026
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6968 - loss: 0.7833 - val_accuracy: 0.9700 - val_loss: 0.3546
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8506 - loss: 0.4408 - val_accuracy: 1.0000 - val_loss: 0.1324
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9287 - loss: 0.2512 - val_accuracy: 1.0000 - val_loss: 0.0513
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9667 - loss: 0.1411 - val_accuracy: 1.0000 - val_loss: 0.0205
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9784 - loss: 0.0894 - val_accuracy: 1.0000 - val_loss: 0.0091
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9855 - loss: 0.0648 - val_accuracy: 1.0000 - val_loss: 0.0063
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9911 - loss: 0.0475 - val_accuracy: 1.0000 - val_

Test Loss: 9.932860848493874e-05
Test Accuracy: 1.0
Model saved as 'a_to_z_supermarket_model.h5'


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import logging

logging.basicConfig(level=logging.INFO)

model_path = "/content/a_to_z_supermarket_model.h5"
model = load_model(model_path)

dataset_path = "a_to_z_supermarket_transactions.csv"
data = pd.read_csv(dataset_path)

label_encoder_customer = LabelEncoder()
data['customer_id'] = label_encoder_customer.fit_transform(data['customer_id'])

label_encoder_product = LabelEncoder()
data['product_id'] = label_encoder_product.fit_transform(data['product_id'])

category_encoder = LabelEncoder()
data['category'] = category_encoder.fit_transform(data['category'])

supplementary_products = {
    "Bread": ["Jam", "Butter", "Cheese"],
    "Milk": ["Sugar", "Coffee", "Tea"],
    "Rice": ["Curry Powder", "Vegetables", "Spices"],
}

default_supplementary_products = {
    "Fruits": ["Vegetables"],
    "Vegetables": ["Fruits"],
    "Snacks": ["Beverages"],
    "Beverages": ["Bakery", "Dairy"],
    "Bakery": ["Snacks"],
    "Dairy": ["Beverages"],
    "Pasta": ["Sauce"],
    "Cereal": ["Milk"],
    "Tea": ["Biscuits"],
    "Coffee": ["Sugar"],
    "Chips": ["Salsa"],
    "Frozen Foods": ["Sauces"],
    "Chicken": ["Spices"],
    "Fish": ["Lemon"],
    "Cheese": ["Crackers"],
    "Yogurt": ["Fruits"],
    "Ice Cream": ["Toppings"],
    "Wine": ["Cheese"],
    "Chocolates": ["Gift Wrapping"],
    "Bread": ["Deli Meats"],
    "Vegetables": ["Dip"],
    "Eggs": ["Bacon"],
    "Spaghetti": ["Meatballs"],
    "Soup": ["Croutons"],
    "Pizza": ["Soft Drinks"],
    "Juices": ["Cookies"],
}

def get_customer_history(customer_id):
    customer_id_encoded = label_encoder_customer.transform([customer_id])[0]
    customer_history = data[data['customer_id'] == customer_id_encoded]
    if customer_history.empty:
        logging.warning(f"No purchase history found for customer ID: {customer_id}")
    return customer_history

def compute_parameters(history):
    total_purchases = len(history)
    favorite_category = history['category'].mode()[0]
    total_spent = history['total_spent'].sum()
    average_quantity = history['quantity'].mean()
    most_expensive_purchase = history['price'].max()
    average_price = history['price'].mean()
    return [total_purchases, favorite_category, total_spent, average_quantity, most_expensive_purchase, average_price]

def recommend_products(customer_id):
    customer_history = get_customer_history(customer_id)
    if customer_history.empty:
        return "No purchase history found for this customer."

    parameters = compute_parameters(customer_history)
    logging.info(f"Computed Parameters: {parameters}")

    scaler = MinMaxScaler()
    parameters_scaled = scaler.fit_transform(np.array(parameters).reshape(1, -1))

    prediction = model.predict(parameters_scaled)
    recommended_category_index = np.argmax(prediction)

    recommended_category = category_encoder.inverse_transform([recommended_category_index])[0]
    logging.info(f"Recommended Category: {recommended_category}")

    supplementary_items = supplementary_products.get(recommended_category, [])
    if not supplementary_items:
        supplementary_items = default_supplementary_products.get(recommended_category, [])

    if supplementary_items:
        logging.info(f"Suggested supplementary products: {', '.join(supplementary_items)}")
    else:
        logging.info("No supplementary products found for the recommended category.")

    return recommended_category, supplementary_items

customer_id = input("Enter Customer ID: ")
recommended_category, supplementary_items = recommend_products(customer_id)
print(f"Recommended Category: {recommended_category}")
print(f"Suggested supplementary products: {', '.join(supplementary_items)}" if supplementary_items else "No supplementary products available.")


Enter Customer ID: C00744
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Recommended Category: Dairy
Suggested supplementary products: Beverages


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
